<a href="https://colab.research.google.com/github/CalipsoAdara/SMN/blob/verif_S2/DownscalingCHIRPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Tema: Downscaling ⚡**

Para poder comparar GFS y CHIRPS es necesario reducir la resolucion del CHIRPS de 0.05°x0.05° a la del GFS (0.5°x0.5°). Antes de realizar el downscaling, guardamos la data del CHIRPS correspondiente al cuadrado de SUDAMERICA.

*0.05° ---> 0.5°*

*osea, necesito **promediar** 10 pixeles de 0.05° para obtener 0.5°*

Como vamos a reducir la resolucion, con la funcion "coarse" de XARRAY, es importante notar dos cosas. La *primera* que es necesario tener dimensiones pares. La *segunda*, que vamos a perder informacion en los bordes. Por ejemplo, como el CHIRPS solo llega hasta 50°S al downscalear vamos a perder medio grado de informacion.

La region de Sudamerica deseada es:

-latitud: de 50°S a 10°S

-longitud: de 90°O a 30°O  (o 270° a 330°)

Considerando eso, y que quiero que los valores al downscalear queden de la forma 90, 89.5, 89, 88.5 ,etc en vez de 90.75, 90.25, 89.75, 89.25, etc, el recorte del archivo chirps se hace: 

-latitud: de 49.75°S a 10.75°S

-longitud: de 90.25°O a 29.75°O 

---



## Instalacion e importar librerias necesarias

In [2]:
# instalar
!pip install xarray
!pip install shapely==1.8.5
!pip install datetime
!pip install matplotlib==3.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 47.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 5.1 MB/s eta 0:00:00
  Created wheel for cartopy: filename=Cartopy-0.21.1-cp38-cp38-linux_x86_64.whl size=11127993 sha256=352c2d42481c441a74fab0387726e93b928a850003b7334426d1fe238c370890
  Stored in directory: /root/.cache/pip/wheels/7c/ff/6f/09e4aa53f4e67d6498ef1d33dd4ea9c8c165817b9ddcc31d28
Successfully built cartopy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 34.3 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.0
    Uninstalling matplotlib-3.7.0:
      Successfully uninstalled matplotlib-3.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
metpy 1.4.0 requires matplotlib>=3.3.0, but you have matplotlib 3.1.3 which is incompatible.


In [3]:
# importar
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import osgeo.ogr

In [4]:
# Montar drive  (esto se elimina al realizar el codigo en el server)
from google.colab import drive
drive.mount('/content/drive')
os.chdir('./drive/MyDrive/Verifsem2')

Mounted at /content/drive


In [ ]:
# Periodo
years = range(1981,2022+1)  # el +1 para que incluya el ultimo valor
meses = range(1,12+1)


# Nombre de los archivos
filelist = ["chirps-v2.0."+str(y) +"."+str(m)+".days_SA.nc" for y in years for m in meses]

In [ ]:
# Downscaling
for f in filelist:
  # Leo con xarray
  ds = xr.open_dataarray(f)

  # DOWNSCALING 
  ds_down = ds.coarsen(latitude=10, longitude=10).mean()

  # Guardo
  ds_sa.to_netcdf(f[0:26] + "_down.nc")